## Example using R in JupyterNootebooks for tranforming Population statistic data files.

For analyse population statisctic uses three sources: Indian Registration System (ITK), Statisctic Canada, Inuit Tapirrit Kanatami (ITK).
For showing an example using R language in JupyterNotebooks we took one source file (ITK) and transformated it for one result CTBRS table
### Step 1. Importing necessary modules
  
Take native R libraries: readxl, writexl

In [11]:
library(readxl)
library(writexl)

### Step 2. Load data from file

In [ ]:
file_path_incoming_data <- "data/INACData.xlsx"
file_path_outgoing_data <- "data/CTBRSResults.xlsx"

full_data <- as.data.frame(
  read_excel(
    file_path_incoming_data, sheet = 2, range = "A6:K195", col_names = FALSE
  )
)

### Step 3. Define default data lines count which belong to a region and define a function of transforming loaded data into a vector

In [13]:
count_values <- 19

get_data <- function(income_data, vector_number, count_lines = NULL) {
  data_set <- c()

  for (i in seq_along(income_data[[vector_number]])) {
    value <- income_data[[vector_number]][i]

    if (is.na(value)) {
      next
    }

    data_set <- c(data_set, value)

    if (!is.null(count_lines) && i == count_lines) {
      return(data_set)
    }
  }

  return(data_set)
}

### Step 4. Creating data vectors (columns)

In [14]:
#Creating data vectors (columns)
regions <- get_data(full_data, 1)
ages <- get_data(full_data, 2, count_values)
total_count <- get_data(full_data, 3)
male_count <- get_data(full_data, 4)
female_count <- get_data(full_data, 5)

### Step 5. Preparing data object in current table format incoming data

In [15]:
structured_data <- list()
index <- 1

for (region in regions) {
  for (age in ages) {
    list <- list()
    list[[age]] <- c(
      total_count[index],
      male_count[index],
      female_count[index]
    )
    structured_data[[ region ]] <- c(structured_data[[ region ]], list)
    index <- index + 1
  }
}

### Step 6. Processing data. As an example we make a one result table in a different format data

In [16]:
ages_column <- c()

for (name_region in names(structured_data)) {
  for (data_region in  structured_data[name_region]) {
    for (name_age in  names(data_region)) {
      ages_column <- c(ages_column, name_age)
    }

    break
  }

  break
}

region_column <- list()

for (name_region in names(structured_data)) {
  amount_column <- c()

  for (data_region in  structured_data[name_region]) {
    for (name_age in  names(data_region)) {
      for (data_age in  data_region[name_age]) {
        amount_column <- c(amount_column, data_age[1])
      }
    }
  }

  region_column[[name_region]] <- amount_column
}

### Step 7. Save data to the result file

In [17]:
data_frame_out <- data.frame(
  `Age group` = c(ages_column)
)

for (region in names(region_column)) {
  data_frame_out <- cbind(data_frame_out, amount = region_column[region])
  names(data_frame_out)[names(data_frame_out) == "amount"] <- region
}

write_xlsx(data_frame_out, file_path_outgoing_data)